# Figure 04
## FOXSI3 Instrumentation Milo's paper
https://v2.overleaf.com/read/wcfwdbgdfgcc
### October 2018
### Milo BC @ UC Berkeley

### Goal:

<div class="alert alert-info">
Another plot show ghost rays as a function of spacing between shells<br/>
<div/>

# Question: What's a good way to show the space between shells dependance of Ghost Rays?

In [3]:
import numpy as np

In [1]:
FrontRadii = [5.345, 5.085, 4.835, 4.595, 4.365, 4.15, 3.94, 3.725, 3.51, 3.29]
InterRadii = [5.151,4.9,4.659,4.429,4.21,4.0,3.799,3.59,3.38,3.17]
InnerRadii = [4.56, 4.335, 4.125, 3.92, 3.725, 3.54, 3.36, 3.18, 2.995, 2.805]

In [9]:
0.5 * np.array(InterRadii)

array([2.5755, 2.45  , 2.3295, 2.2145, 2.105 , 2.    , 1.8995, 1.795 ,
       1.69  , 1.585 ])

In [6]:
2.0 * np.array(InterRadii)

array([10.302,  9.8  ,  9.318,  8.858,  8.42 ,  8.   ,  7.598,  7.18 ,
        6.76 ,  6.34 ])

### Save this analysis for later